### 데이터 셋 만들기
[주제별 텍스트 일상 대화 데이터](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=543)

- 주제 분포

|주제|건수|사용여부|
|:--:|:--:|:----:|
|식음료|5,176|O|
|주거와 생활|5,439|X|
|교통|5,614|X|
|회사/아르바이트|5,550|O|
|군대|5,255|X|
|교육|5,680|O|
|가족|5,599|O|
|연애/결혼|5,461|O|
|반려동물|5,322|O|
|스포츠/레저|5,731|O|
|게임|5,467|X|
|여행|5,550|O|
|계절/날씨|5,438|X|
|사회이슈|5,338|X|
|타 국가 이슈|5,072|X|
|미용|5,775|O|
|건강|5,534|X|
|상거래 전반|5,591|X|
|방송/연예|5,425|X|
|영화/만화|5,597|O|

- train: `/TL` 로 시작하는 파일
- valid: `/VL` 로 시작하는 파일

In [ ]:
# !pip install -q soynlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 KB 7.5 MB/s eta 0:00:00


In [ ]:
import os
from glob import glob
from zipfile import ZipFile

import json
from tqdm import tqdm

import numpy as np
import pandas as pd

import re
from soynlp.normalizer import repeat_normalize

In [ ]:
# train data zip
train_fnames = glob('/content/TL*.zip')
print('train zip: \n', train_fnames)

# validation data zip
valid_fnames = glob('/content/VL*.zip')
print('validation zip: \n', valid_fnames)

train zip: 
 ['/content/TL_01. KAKAO(3).zip', '/content/TL_03. INSTAGRAM.zip', '/content/TL_01. KAKAO(1).zip', '/content/TL_04. BAND.zip', '/content/TL_01. KAKAO(4).zip', '/content/TL_05. NATEON.zip', '/content/TL_02. FACEBOOK.zip', '/content/TL_01. KAKAO(2).zip']
validation zip: 
 ['/content/VL_05. NATEON.zip', '/content/VL_02. FACEBOOK.zip', '/content/VL_04. BAND.zip', '/content/VL_01. KAKAO.zip', '/content/VL_03. INSTAGRAM.zip']


In [ ]:
def make_unzip(path: str, fnames):
    # 폴더 생성
    print(f'Create a {path} dataset folder')
    if not os.path.isdir(path):
        os.mkdir(path)
    
    # 압축 해제
    for fname in fnames:
        with ZipFile(fname, 'r') as zip:
            zip.extractall(path)
            print(f'File is unzipped in {path} folder')

In [ ]:
make_unzip('train', train_fnames)

Create a train dataset folder
File is unzipped in train folder
File is unzipped in train folder
File is unzipped in train folder
File is unzipped in train folder
File is unzipped in train folder
File is unzipped in train folder
File is unzipped in train folder
File is unzipped in train folder


In [ ]:
make_unzip('valid', valid_fnames)

Create a valid dataset folder
File is unzipped in valid folder
File is unzipped in valid folder
File is unzipped in valid folder
File is unzipped in valid folder
File is unzipped in valid folder


In [ ]:
train_jsons = glob('/content/train/*.json')
print('train json file sample: \n', train_jsons[:3])

valid_jsons = glob('/content/valid/*.json')
print('valid json file sample: \n', valid_jsons[:3])

train json file sample: 
 ['/content/train/INSTAGRAM_142_19.json', '/content/train/KAKAO_3423_20.json', '/content/train/KAKAO_3075_07.json']
valid json file sample: 
 ['/content/valid/FACEBOOK_96_17.json', '/content/valid/KAKAO_627_12.json', '/content/valid/KAKAO_779_20.json']


데이터셋 만들기

In [ ]:
def make_csv(fnames):
    text = []
    label = []
    
    for fname in tqdm(fnames):
        f = open(fname, encoding="utf-8")
        try:
            data = json.loads(f.read())
            annotation = data['info'][0]['annotations']
            text.append(annotation['text'])
            label.append(annotation['subject'])
        except:
            print(f"\n**{fname}: 오류 파일**\n")
    
    df = pd.DataFrame({'text': text, 'label': label})
    
    return df

In [ ]:
pattern = re.compile(r'\d+\s:\s')
text_pattern = re.compile(r'[^ A-Za-z0-9가-힣+]')

def cleaning(x):
    x = re.sub("\n", " ", x)
    x = pattern.sub('', x)
    x = text_pattern.sub('', x)
    x = repeat_normalize(x, num_repeats=2)
    
    return x

train dataset
- 오류 파일 존재: `/content/train/KAKAO_1648_13.json`

In [ ]:
train_df = make_csv(train_jsons)
train_df.head()

 80%|████████  | 70464/87690 [00:11<00:04, 3889.74it/s]


**/content/train/KAKAO_1648_13.json: 오류 파일**



100%|██████████| 87690/87690 [00:15<00:00, 5635.34it/s]


,text,label
0,1 : 이영애 새 드라마 보신 분\n2 : 이영애 새 드라마 나와?\n3 : 난 방...,방송/연예
1,1 : 엄마 10월 그리고리 영화봤어요?\n2 : 아니 그게 뭔지 몰라\n1 : 죽...,영화/만화
2,1 : 내가 곰곰이 생각해 보니 우리 가족은 웃음이 없는 것 같아요\n2 : 나도 ...,가족
3,1 : 너 인공눈물 좀 있냐?\n2 : 아니 나도 얼마 없어\n1 : 아 전에 병원...,건강
4,1 : 당근마켓이나 중고나라에서 거래해 본 적 있나? 키키\n2 : 응응 해봤어 키...,상거래 전반


In [ ]:
train_df['text'] = train_df.apply(lambda x: cleaning(x['text']), axis=1)
train_df.head()

,text,label
0,이영애 새 드라마 보신 분 이영애 새 드라마 나와 난 방송은 못 믿겠더라 맛집 소개...,방송/연예
1,엄마 10월 그리고리 영화봤어요 아니 그게 뭔지 몰라 죽기 전에 봐야 할 영화 2번...,영화/만화
2,내가 곰곰이 생각해 보니 우리 가족은 웃음이 없는 것 같아요 나도 그렇게 생각했는데...,가족
3,너 인공눈물 좀 있냐 아니 나도 얼마 없어 아 전에 병원가서 좀 받아둘걸 얼마전에 ...,건강
4,당근마켓이나 중고나라에서 거래해 본 적 있나 키키 응응 해봤어 키키 최근에 당근마켓...,상거래 전반


In [ ]:
# 데이터 저장
train_df.to_csv('/content/train.csv', index=False, encoding='utf-8')

validation dataset

In [ ]:
valid_df = make_csv(valid_jsons)
valid_df.head()

100%|██████████| 10962/10962 [00:03<00:00, 3304.89it/s]


,text,label
0,1 : 언니 지금 비타민 챙겨 먹고 있어?\n2 : 웅 챙겨 먹어\n2 : 나이가 ...,건강
1,1 : 여행사 껴서 여행 가는 거 생각보다 괜찮더라\n2 : 맞아~ 나도 가끔 가는...,여행
2,"1 : 재밌는 영화 추천 좀 해봐 키키\n2 : 요새 영화 안 본지 오래됨,\n1 ...",영화/만화
3,1 : 오빠~ 오늘 몇 시에 와?\n2 : 하… 자기야 오빠 오늘 회식이래…\n1 ...,회사/아르바이트
4,1 : 작은 아빠랑 작은 엄마 다들 주사 맞았어?\n2 : 아 그거 백신 말하는 건...,가족


In [ ]:
valid_df['text'] = valid_df.apply(lambda x: cleaning(x['text']), axis=1)
valid_df.head()

,text,label
0,언니 지금 비타민 챙겨 먹고 있어 웅 챙겨 먹어 나이가 있어서 맞아 20대 때부터 ...,건강
1,여행사 껴서 여행 가는 거 생각보다 괜찮더라 맞아 나도 가끔 가는데 일정 생각 안하...,여행
2,재밌는 영화 추천 좀 해봐 키키 요새 영화 안 본지 오래됨 나두 그래 너무 재밌는 ...,영화/만화
3,오빠 오늘 몇 시에 와 하 자기야 오빠 오늘 회식이래 갑자기 오늘 바로 회식 한대 ...,회사/아르바이트
4,작은 아빠랑 작은 엄마 다들 주사 맞았어 아 그거 백신 말하는 건가 응 다들 1차는...,가족


In [ ]:
# 데이터 저장
valid_df.to_csv('/content/valid.csv', index=False, encoding='utf-8')